In [11]:
import pandas as pd
import numpy as np

import re
import torch
import faiss

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

from bleurt import score

In [2]:
df = pd.read_csv('./df_data.csv')
df

,text
0,\nКапитан сборной Аргентины Лионель Месси сооб...
1,\nЛидер сборной Аргентины Лионель Месси вряд л...
2,\nINTERFAX.RU - Аргентинский футболист Лионель...
3,\nЛионель Месси не планирует участвовать в чем...
4,\nКапитан сборной Аргентины по футболу Лионель...
...,...
95,\nТинькофф Банк объявил о запуске Центрального...
96,\n«Тинькофф» запускает частный университет со ...
97,\n14 июня 2023 года «Тинькофф» объявил о запус...
98,\nТинькофф банк запустит частный университет -...


In [3]:
# Задаем путь к предобученной модели

model_ckpt = "papluca/xlm-roberta-base-language-detection"

In [4]:
# Инициализируем токенизатор и модель, а также функции, которые возвращают эмбеддинги

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(end='')

Some weights of the model checkpoint at papluca/xlm-roberta-base-language-detection were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [5]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]


def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, 
                              truncation=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    return cls_pooling(model_output)


# Посчитаем эмбединги в специальной структуре
# Определять 'близость' запроса и темы из датасета будем по эмбеддингам тэгов 

ds = Dataset.from_pandas(df)
embeddings_dataset = ds.map(
    lambda x: {'text_embeddings': get_embeddings(x['text']).detach().cpu().numpy()[0]})
embeddings_dataset.add_faiss_index(column='text_embeddings')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'text_embeddings'],
    num_rows: 100
})

In [6]:
# Создадим функцию, которая вернет 'близкие' 10 текстов

def get_sequences(question):
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        'text_embeddings', question_embedding, k=10)
    return samples['text']

In [7]:
# Зададим вопрос и получим упорядоченный список от наиболее вероятных к наименее

question = 'университет тинькофф'
base_articles = get_sequences(question)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

In [10]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


input_ids = tokenizer(
    [WHITESPACE_HANDLER(el) for el in base_articles],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=1024
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    min_length=700,
    no_repeat_ngram_size=3,
    num_beams=4,
    max_new_tokens=900
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Российская компания «Тинькофф» запускает частный университет со специализацией на математике и компьютерных науках. Программа обучения стартует в сентябре 2024 года, а магистерская программа «Академия Data Science» начнется в июне 2023 года, сообщает корреспондент Русской службы Би-би-си Ольга Яресько. Это уже вторая часть проекта компании, которая будет работать в России в течение нескольких лет. Что будет с программой обучения.   . . В bbc.co.uk ..  Эти и інші - в bbc .com  , . BBC News  ( bbcrussian.com. ) . "Тинькоff , "Тинкофф", а уже в этом году, когда уже можно будет понять, как будет развиваться этот университет, и почему в России не хватает студентов, которые будут участвовать в программе обучения в ближайшие годы и какие будут новые образовательные программы для студентов в следующем году, рассказывает корреспондент ВВС Украина , что будет на пути к этому пути? bbcuzbek.com рассказывает о том, как будут развиваться эти и другие сферы обучения, о которых рассказывает Русская с

In [26]:
checkpoint = "./bleurt/bleurt/test_checkpoint"
references = ['''\n«Тинькофф» запускает частный университет со специализацией на математике и компьютерных науках\n\n«Тинькофф» запускает «Центральный университет». Это первый в России частный университет, который основан на модели обучения STEM (Science, Technology, Engineering, Mathematics), которая объединяет естественные и инженерные предметы в рамках единой системы. На первом этапе программа обучения включает следующие дисциплины: программная инженерия; компьютерная безопасность; бизнес-информатика; прикладная математика; анализ данных и искусственный интеллект; экономика и анализ данных; маркетинг и бизнес-аналитика; продакт-менеджмент. «Тинькофф» обещает обновлять программу каждый год. В её формировании примут участие крупные российские работодатели и «известные на международном уровне специалисты академического мира». По окончании обучения студенты получат диплом государственного образца по направлению «Математика и компьютерные науки» (02.03.01). Вскоре «Тинькофф» разработает образовательные программы и по другим сферам, включая химию, биологию и естественные науки. Программа бакалавриата стартует в сентябре 2024 года. В первый набор войдут 500 студентов и 55 преподавателей. В сентябре 2023. года запустится магистерская программа «Академия Data Science», которая рассчитана на два года обучения. Стоимость обучения составит от 70 000 рублей в месяц. Студентам предложат гранты или кредит со ставкой 3% годовых и ежемесячным платежом до 3000 рублей в месяц. Обучение будет происходить в Москве. Но «Тинькофф» обещает открыть филиалы «Центрального университета» в других городах России в течение нескольких лет. А до 2026 года компания построит общежитие, которое вместит до 5000 студентов. \n''']
candidates = ['''Российская компания «Тинькофф» запускает частный университет со специализацией на математике и компьютерных науках. Программа обучения стартует в сентябре 2024 года, а магистерская программа «Академия Data Science» начнется в июне 2023 года, сообщает корреспондент Русской службы Би-би-си Ольга Яресько. Это уже вторая часть проекта компании, которая будет работать в России в течение нескольких лет. Что будет с программой обучения.   . . В bbc.co.uk ..  Эти и інші - в bbc .com  , . BBC News  ( bbcrussian.com. ) . "Тинькоff , "Тинкофф", а уже в этом году, когда уже можно будет понять, как будет развиваться этот университет, и почему в России не хватает студентов, которые будут участвовать в программе обучения в ближайшие годы и какие будут новые образовательные программы для студентов в следующем году, рассказывает корреспондент ВВС Украина , что будет на пути к этому пути? bbcuzbek.com рассказывает о том, как будут развиваться эти и другие сферы обучения, о которых рассказывает Русская служба Би-Би-Си , в том числе и в России, где будет обучение и на каких условиях будут работать российские студенты, которым будет заниматься предпринимательство и что может быть связано с этим опытом? Вот уже несколько дней, кто будет изучать образовательную программу и зачем будет платить за обучение в России? - об этом сообщили в пресс-службе компании. Вот несколько самых интересных научных новостей о программе компании и бизнесе в российской экономике, которую будет проводить компания, пытаясь получить возможность получить диплом государственного образца в 2024 году, чтобы повысить эффективность образовательной системы и не только в вузах, но именно - о перспективах российского университета, который начнет выпускать в Москве и Санкт-Петербурге, куда они начнут работать уже через несколько лет, сколько будет стоить для них и чем будет учиться в России. Чем заняты студенты - это и самые перспективные и интересные новости о будущих студентах и финансовых возможностях и перспективах, если они не будут заняты в этой сфере, как выяснил корреспондент BBC Capital , посвященные этому вопросу, может стать одним из самых популярных в мире. bbc Русской православной церкви, считает корреспондент Би-си , какие перспективы имеет российская компания, которой уже занимаются образовательными программами и программами, которыми будут заниматься студенты в одной из крупнейших российских университетов, которых ожидается уже сейчас, - чтобы получить деньги в образовательных программах, оказались в одном из них, например, в Петербурге , некоторые из которых, по ее словам, будет запущена новая программа обучения .''']

scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references=references, candidates=candidates)
assert isinstance(scores, list) and len(scores) == 1
print(scores)

INFO:tensorflow:Reading checkpoint ./bleurt/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
[0.012895867228507996]


In [34]:
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize

In [51]:
tokenized_summary = word_tokenize(summary, language="russian")

In [52]:
tokenized_articles = word_tokenize(base_articles[0])

In [43]:
round(meteor_score.meteor_score([tokenized_articles], tokenized_summary), 4)

0.2314

In [44]:
def token_articles(base_articles):
    tokenized_articles = []
    for i in range(len(base_articles)):
        tokenized_articles.append(word_tokenize(base_articles[i]))
    return tokenized_articles

In [47]:
tokenized_articles = token_articles(base_articles)

In [50]:
round(meteor_score.meteor_score(tokenized_articles, tokenized_summary), 4)

0.2356